- TF-IDF : 텍스트 벡터화
- PCA : 차원 축소
- LSA : 잠재 의미 분석
- t-SNE : 2D시각화
- 로지스틱회귀
- 토큰화 & 전처리

In [1]:
# LSA
# TF-IDF 행렬에 대해서 SVD 를 적용
# 단어와 문서 간의 숨겨진 의미 관계를 찾음
# PCA 차이:
    # PCA : 데이터 자체 분산 최대화
    # LSA : 문서-단어형태의 의미 구조 파악
# "은행"
    # "돈"  "계좌"     주변에 등장
    # "나무" "냄세" "먹는다" 주변에 등장

In [2]:
# t-SNE : 고차원 데이터를 2D/3D로 변환 - 시각화 전용(분석에는 부적합), 계산이 오래걸림
# PCA vs t-SNE  
# PCA : 속도가 빠름, 전역 구조 보존
# t-SNE : 느림, 국소(지역) 군집 명확

In [3]:
# 데이터셋
from sklearn.datasets import load_files
train_path = r'C:\LLM\ch05\20newsbydate\20news-bydate-train'
test_path = r'C:\LLM\ch05\20newsbydate\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')
categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

x_train = [ clean_text(t) for t in train_data]
x_test = [ clean_text(t) for t in test_data]
y_train = train_target
y_test = test_target

In [6]:
# 텍스트 전처리 ( 소문자 + 토큰화(3글자이상) + 불용어제거(stopwords) + 어간추출(stemming))  --> 영어
# 파이프라인 
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer  # 같은 의미의 다른형태 단어를 통일
from sklearn.feature_extraction.text import TfidfVectorizer
regtok = RegexpTokenizer(r"[\w']{3,}")
english_stops = set(stopwords.words('english'))
# 커스텀 토크나이져
def tokenizer(text):
    tokens = regtok.tokenize(text)
    words = [word for word in tokens if word not in english_stops]
    features = list(map(lambda x : PorterStemmer().stem(x), words))
    return features
# TF-IDF 벡터화
tfidf = TfidfVectorizer(tokenizer=tokenizer,max_features=2000,min_df=2,max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
# 분류모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=200,random_state=42)
lr_clf.fit(x_train_tfidf,y_train)

print('기본분류------------')
print(f'학습정확도 : {lr_clf.score(x_train_tfidf,y_train)}')
print(f'테스트정확도 : {lr_clf.score(x_test_tfidf,y_test)}')

print('주성분분석')
# 데이터의 분산이 가장 큰 방향
# 선형번환만 가능
# 모든 데이터의 특성을 평등하게 고려
# 용도 : 시각화, 속도 개선
# 2000차원 - >100차원축소
from sklearn.decomposition import PCA
pca = PCA(n_components=100,random_state=42)
x_train_pca = pca.fit_transform(x_train_tfidf.toarray())
x_test_pca = pca.transform(x_test_tfidf.toarray())

import numpy as np
cumsum_var =  np.cumsum( pca.explained_variance_ratio_)
print(f'원본 차원 : {x_train_tfidf.shape[1]}')
print(f'축소후 차원 : {x_train_pca.shape[1]}')
print(f'설명된 분산 : {pca.explained_variance_.sum()}')
print(f'누적 분산 : {cumsum_var[:10]}')

# pca 후 분류
lr_clf_pca = LogisticRegression(max_iter=200,random_state=42)
lr_clf_pca.fit(x_train_pca,y_train)

print('주성분 분석 분류------------')
print(f'학습정확도 : {lr_clf_pca.score(x_train_pca,y_train)}')
print(f'테스트정확도 : {lr_clf_pca.score(x_test_pca,y_test)}')

c:\Users\playdata2\miniconda3\envs\LLM\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


기본분류------------
학습정확도 : 0.9705014749262537
테스트정확도 : 0.8174427198817442
주성분분석
원본 차원 : 2000
축소후 차원 : 100
설명된 분산 : 0.3282151895915271
누적 분산 : [0.01211923 0.02229888 0.03189694 0.04138118 0.0493389  0.05676164
 0.0635188  0.0701598  0.0765997  0.0826845 ]
주성분 분석 분류------------
학습정확도 : 0.9011799410029498
테스트정확도 : 0.7923133776792314
